In [328]:
import sqlite3
import pandas as pd

In [329]:
con = sqlite3.connect(':memory:')

In [330]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [331]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [332]:
pd.read_sql(
    '''
    SELECT * FROM Customers
    ''',
    con,
)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,5023,Mexico
3,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK
4,5,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,S-958 22,Sweden
...,...,...,...,...,...,...,...
86,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland
87,88,Wellington Importadora,Paula Parente,"Rua do Mercado, 12",Resende,08737-363,Brazil
88,89,White Clover Markets,Karl Jablonski,305 - 14th Ave. S. Suite 3B,Seattle,98128,USA
89,90,Wilman Kala,Matti Karttunen,Keskuskatu 45,Helsinki,21240,Finland


In [333]:
tasks = 0

In [334]:
#1.Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address    
    FROM Customers                           
    where Country in ('Germany', 'France') or City = 'Madrid'  
    order by Country
    ''',
    con,
)

,CustomerName,Country,Address
0,Blondel père et fils,France,"24, place Kléber"
1,Bon app',France,"12, rue des Bouchers"
2,Du monde entier,France,"67, rue des Cinquante Otages"
3,Folies gourmandes,France,"184, chaussée de Tournai"
4,France restauration,France,"54, rue Royale"
5,La corne d'abondance,France,"67, avenue de l'Europe"
6,La maison d'Asie,France,1 rue Alsace-Lorraine
7,Paris spécialités,France,"265, boulevard Charonne"
8,Spécialités du monde,France,"25, rue Lauriston"
9,Victuailles en stock,France,"2, rue du Commerce"


In [335]:
#2.Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

tasks+=1
pd.read_sql(
    '''
    SELECT country, COUNT(CustomerName) AS customer_count
    FROM customers
    GROUP BY country
    ORDER BY customer_count DESC
    LIMIT 3;
    ''',
    con,
)

,Country,customer_count
0,USA,13
1,Germany,11
2,France,11


In [336]:
#3.Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

tasks+=1
pd.read_sql(
    '''
    SELECT ShipperName, OrderDate
    FROM (
    SELECT ShipperName, OrderDate, ROW_NUMBER() OVER (ORDER BY OrderDate) AS row_num
    FROM Shippers S left join Orders O on S.ShipperID=O.ShipperID
    ) AS ordered_orders
    WHERE row_num = 10;
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,02.10.1996


In [337]:
#4.Выберите самый дорогой заказ, выведите список товаров с их ценами.

tasks+=1
pd.read_sql(
    '''
    SELECT ProductName, (quantity * price) as total_price
    FROM Products P left join OrderDetails O on P.ProductID=O.ProductID
    GROUP BY ProductName
    ORDER BY total_price desc    
    ''',
    con,
)

#LIMIT 1 - не стал применять так как задача сформлирована не достаточно точно

,ProductName,total_price
0,Northwoods Cranberry Sauce,2800.0
1,Côte de Blaye,2635.0
2,Mishi Kobe Niku,1940.0
3,Chocolade,892.5
4,Schoggi Schokolade,658.5
...,...,...
72,Gravad lax,26.0
73,Spegesild,24.0
74,Geitost,20.0
75,Konbu,12.0


In [338]:
#5.Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

tasks+=1
pd.read_sql(
    '''
    SELECT  ProductName, sum(Quantity) as Q
    FROM Products P left join OrderDetails O on P.ProductID=O.ProductID
    GROUP BY ProductName
    ORDER BY Q desc
    
    ''',
    con,
)


,ProductName,Q
0,Gorgonzola Telino,458
1,Camembert Pierrot,430
2,Steeleye Stout,369
3,Raclette Courdavault,346
4,Chang,341
...,...,...
72,Genen Shouyu,25
73,Mishi Kobe Niku,20
74,Gustaf's Knäckebröd,18
75,Røgede sild,15


In [339]:
#6.Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

tasks+=1
pd.read_sql(
    '''
    SELECT SupplierName,  Country, ContactName, Phone
    FROM Suppliers S left join Products P on S.SupplierID=P.SupplierID
    left join OrderDetails OD on P.ProductID=OD.ProductID
    GROUP BY SupplierName
    HAVING count(Quantity)
    ORDER BY SupplierName desc
    LIMIT 5;
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,Zaanse Snoepfabriek,Netherlands,Dirk Luchte,(12345) 1212
1,Tokyo Traders,Japan,Yoshi Nagase,(03) 3555-5011
2,Svensk Sjöföda AB,Sweden,Michael Björn,08-123 45 67
3,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
4,Refrescos Americanas LTDA,Brazil,Carlos Diaz,(11) 555 4640


In [344]:
#7.Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

tasks+=1
pd.read_sql(
    '''
    SELECT Country, CategoryName, Price
    FROM Categories  left join Products on Categories.CategoryID=Products.CategoryID
    left join OrderDetails on Products.ProductID=OrderDetails.ProductID
    left join Orders on OrderDetails.OrderID=Orders.OrderID
    left join Customers on Orders.CustomerID=Customers.CustomerID
    where Country = 'Brazil'
    GROUP BY CategoryName
    HAVING max(Price)
    ORDER BY Price desc
    ''',
    con,
)

,Country,CategoryName,Price
0,Brazil,Beverages,263.50
1,Brazil,Meat/Poultry,123.79
2,Brazil,Confections,81.00
3,Brazil,Dairy Products,55.00
4,Brazil,Produce,53.00
5,Brazil,Condiments,43.90
6,Brazil,Seafood,19.00


In [340]:
#8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

tasks+=1
pd.read_sql(
    '''
   
   WITH OP AS (
    SELECT Orders.OrderID, SUM(OrderDetails.quantity * Products.price) AS total_price
    FROM Categories left join Products on Categories.CategoryID=Products.CategoryID
    left join OrderDetails on Products.ProductID=OrderDetails.ProductID
    left join Orders on OrderDetails.OrderID=Orders.OrderID
    left join Customers on Orders.CustomerID=Customers.CustomerID
    where Country = 'USA'
    GROUP BY Orders.OrderID
),
MaxMinPrices AS (
    SELECT MAX(total_price) AS max_price, MIN(total_price) AS min_price
    FROM OP
)

SELECT (max_price - min_price) AS price_difference
FROM MaxMinPrices;
   
    ''',
    con,
)

,price_difference
0,7638.45


In [352]:
#9.Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

tasks+=1
pd.read_sql(
    '''
    WITH YoungestEmployees AS (
    select BirthDate
    from Employees
    order by BirthDate desc
    LIMIT 3
)

SELECT COUNT(Quantity) AS Quantity_count, LastName|| ' ' || FirstName AS full_name
 FROM Employees E left join Orders O on E.EmployeeID=O.EmployeeID 
 left join OrderDetails OD on O.OrderID=OD.OrderID
GROUP BY Quantity
having COUNT(Quantity)
ORDER BY full_name desc
;
   
    ''',
    con,
)


#не могу понять, в чем дело с датами рождения

,Quantity_count,full_name
0,1,Peacock Margaret
1,2,Peacock Margaret
2,3,Peacock Margaret
3,3,Leverling Janet
4,1,Leverling Janet
5,7,Leverling Janet
6,1,King Robert
7,1,Fuller Andrew
8,5,Fuller Andrew
9,8,Fuller Andrew


In [345]:
#10.Сколько банок крабового мяса всего было заказано.

tasks+=1
pd.read_sql(
    '''
    SELECT ProductName, sum(Quantity)
    FROM Products left join OrderDetails on Products.ProductID=OrderDetails.ProductID
    where ProductName = 'Boston Crab Meat'
  
    ''',
    con,
)


,ProductName,sum(Quantity)
0,Boston Crab Meat,256
